## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-07-06-51-00 +0000,wsj,Ten-Year U.S. Treasury Yield Seen Below 4% in ...,https://www.wsj.com/finance/investing/jgb-futu...
1,2025-11-07-06-17-28 +0000,nyt,San Francisco Sees Nancy Pelosi as a Tireless ...,https://www.nytimes.com/2025/11/06/us/nancy-pe...
2,2025-11-07-06-17-27 +0000,wapo,Philippine death toll from devastating typhoon...,https://www.washingtonpost.com/world/2025/11/0...
3,2025-11-07-06-17-19 +0000,nypost,Massive fiery car explosion that injured 7 fir...,https://nypost.com/2025/11/07/us-news/nyc-car-...
4,2025-11-07-06-14-58 +0000,nyt,"Former King of Spain, Juan Carlos, Details Dea...",https://www.nytimes.com/2025/11/07/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2341/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
127,trump,45
1,year,17
52,new,15
231,will,14
115,shutdown,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
47,2025-11-07-02-15-00 +0000,wsj,"Republican Rep. Elise Stefanik, an ally of Pre...",https://www.wsj.com/politics/elections/elise-s...,130
57,2025-11-07-02-00-00 +0000,wsj,Allies of President Trump and Health Secretary...,https://www.wsj.com/politics/elections/trump-r...,106
264,2025-11-06-17-38-00 +0000,wsj,Many GOP lawmakers are saying no—for now—to Pr...,https://www.wsj.com/politics/policy/senate-rep...,101
196,2025-11-06-20-54-00 +0000,wsj,Employers said they cut more than a million jo...,https://www.wsj.com/economy/jobs/planned-u-s-j...,97
195,2025-11-06-20-57-06 +0000,latimes,Supreme Court rules Trump may remove transgend...,https://www.latimes.com/politics/story/2025-11...,96


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
47,130,2025-11-07-02-15-00 +0000,wsj,"Republican Rep. Elise Stefanik, an ally of Pre...",https://www.wsj.com/politics/elections/elise-s...
163,67,2025-11-06-22-09-16 +0000,nypost,Tesla shareholders approve Elon Musk’s histori...,https://nypost.com/2025/11/06/business/tesla-s...
80,58,2025-11-07-01-18-00 +0000,wsj,Senate Majority Leader John Thune expects Sena...,https://www.wsj.com/politics/policy/government...
188,51,2025-11-06-21-16-00 +0000,wsj,The Bank of Mexico extended its monetary easin...,https://www.wsj.com/articles/bank-of-mexico-cu...
207,50,2025-11-06-20-38-34 +0000,nypost,Congress may not be Pelosi-free for long despi...,https://nypost.com/2025/11/06/us-news/congress...
40,49,2025-11-07-02-31-35 +0000,nypost,James Carville says he’d bet money Dems will w...,https://nypost.com/2025/11/06/us-news/james-ca...
94,43,2025-11-07-00-55-36 +0000,nypost,Lefty Zohran Mamdani ally duking it out with p...,https://nypost.com/2025/11/06/us-news/lefty-zo...
196,42,2025-11-06-20-54-00 +0000,wsj,Employers said they cut more than a million jo...,https://www.wsj.com/economy/jobs/planned-u-s-j...
76,41,2025-11-07-01-28-00 +0000,wsj,Trump Administration Must Fund Full SNAP Benef...,https://www.wsj.com/politics/policy/trump-admi...
289,40,2025-11-06-16-16-32 +0000,latimes,"Up to 1,800 flights a day could be disrupted b...",https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
